# JUAS Accelerator Design Workshop

# Topic III: Lattice Design

The goal of the accelerator design workshop is to apply the knowledge gained during this school to a realistic
study case. The task is to design a particle accelerator with certain specifications and boundary conditions.
The idea is to gain experience how such a big task is tackled and organised. There are twelve groups of three
to four students each. Three topics are assigned, each treats the same problem from a slightly different angle
and with different emphasis.

**Scope: design a top factory for precision measurements.**
Design a particle collider (the *JUAS Collider*) for precision measurements of the top quark mass at the $t\bar{t}$-threshold.
The circumference must not exceed 100 km and the maximum synchrotron radiation power is limited to 50 MW *per beam*.
Per year at least 100000 $t\bar{t}$ pairs should be produced for sufficient statistics.

*Hint:* Should your calculation crash, it is useful to restart the notebook kernel and re-run all previous cells (`Run` $\rightarrow$ `Run All Above Selected Cell`).

## Getting Started

In [ ]:
import numpy as np

import sys, os

import xtrack as xt

from matplotlib import pyplot as plt

%matplotlib inline

*Comment:* You find the xsuite documentation here: https://xsuite.readthedocs.io/en/latest/

## Exercise summary

1) Design basic cell according to beam requirements<br />
2) Implement an xsuite model of your basic cell, close the ring and calculate SR integrals and equilibrium beam parameters<br />
3) Include dispersion suppressors and straight sections to your model and match optics<br />
4) Optional: Include RF cavities and calculate equilibrium beam parameters with xsuite

### Note: this notebook embeds the structure to solve the exercises. 

$\implies$ You need to fill in the gaps indicated with `# fill in here` to make it work.

Also note that, at the end of the course, you are expected to present a well-prepared set of slides during the examination, suitable to show to the other groups working on different topics. You may use this notebook to compute all the results -- however, the notebook itself should <b><u>not</u></b> be used in the presentation.

## Assume following boundary conditions

- A circumference of 100 km<br />
- Maximum synchrotron radiation power: $P = 50$ MW per beam<br />
- The basic cell should have the same phase advance in both planes.<br />
- Damping partition number: $J_x =1$<br />

## 1. Design of basic arc cell

### <span style="color:blue;">a) What lattice type do you choose, and why?</span>

### <span style="color:blue;">b) What phase advance per cell do you choose, and why?</span>

In [ ]:
# phase advance per cell in radian
mu_x = # fill in here
mu_y = # fill in here

### <span style="color:blue;">c) Think  about  the  layout  of  your  basic  cell:  cell  length,  length  of  magnetic  elements,  dipole  filling factor.</span>

*Hint:*  Start with a cell length of 50 m. You should obtain a dipole filling factor of about 80%.

In [ ]:
Lcell = 50  # cell length in m
LB = # fill in here # length of bending magnets in m
LQ = # fill in here # length of quadrupole magnets in m
LS = # fill in here # length of sextupole magnets in m

### <span style="color:blue;">d) Calculate the equilibrium emittance for your lattice.</span>

The equilibrium emittance can be approximately calculated using following equation:

$$\epsilon_x = %\frac{C_{\text{q}}}{J_x} \gamma^2\theta^3 \frac{\rho^2}{l_{\text{B}^3}} \langle\mathcal{H}\rangle = 
\frac{C_{\text{q}}}{J_x} \gamma^2\theta^3 F$$

$C_{\text{q}} = \frac{55}{32\sqrt{3}}\frac{\hbar c}{m_0c^2}=3.832\times10^{-13}$ m, $J_x\approx 1$ is the damping partition number, $\gamma$ the Lorentz factor (discuss with Topic I about the beam energy), and $\theta$ the bending angle of all dipole magnets in a half-cell. For a FODO cell with phase advance $\mu$ the factor $F$ can be written as 

$$F_{\text{FODO}} = \frac{1}{2 \sin\mu} \frac{5+3\cos\mu}{1-\cos\mu} \frac{L}{l_{\text{B}}}.$$

$L$ is the cell length and $l_{\text{B}}$ is the length of all dipole magnets in the cell.

Calculate the bending radius and bending angle of the dipoles assuming 80% of your lattice is filled with arc cells and 20% with straight sections.

In [ ]:
Ebeam = # fill in here # beam energy in [eV] -- discuss with Topic I

rhoB = # fill in here # bending radius of dipoless in [m]
THB = # fill in here # bending angle of dipoles in [rad]
J_x = 1
C_q = 3.832e-13  # in [m]
gamma_L = # fill in here # Lorentz factor

F = # fill in here
epsilon_x = # fill in here  # in [m.rad]

print (epsilon_x)

### <span style="color:blue;">e) Calculate the quadrupole strength $k_1$ using </span>
$$\sin(\mu/2) = \frac{L}{4f} \qquad \text{and}\qquad \frac{1}{f} = k_1 L_{\text{Q}},$$

where $L_Q$ is the length of the quadrupole magnets.

In [ ]:
k1 = # fill in here # in [1/m^2]
print (k1)

### <span style="color:blue;">f) Define the elements and the basic cell in xsuite</span>

Simply execute the following cells:

In [ ]:
# Create an environment
env = xt.Environment()

In [ ]:
# Define variables
env['lb'] = LB  # length of bending magnets in [m]
env['lq'] = LQ  # length of quadrupoles in [m]
env['ls'] = LS  # length of sextupoles in [m]
env['lcell'] = Lcell  # length of basic cell in [m]

In [ ]:
print (f'lcell = {env.vv['lcell']} m')

In [ ]:
env['thb'] = THB
env['k1qf'] = # fill in here
env['k1qd'] = # fill in here
env['k2sf'] = 0.0
env['k2sd'] = 0.0

Definition of the elements -- we need `xt.Bend`, `xt.Quadrupole`, and `xt.Sextupole` magnets:

Example: `env.new('mb', xt.Bend, length='lb', angle='thb')`

xsuite element syntax can be checked in the [xsuite manual](https://xsuite.readthedocs.io/en/latest/).

In [ ]:
env.new('mb', # fill in here )
env.new('mqf', # fill in here )
env.new('mqd', # fill in here )
env.new('msf', # fill in here )
env.new('msd', # fill in here )

Sequence definition of arc cell:

*Hint:* Cut the first quadrupole in half (i.e. define quadrupoles with half the length) and put one at the beginning and one at the end of your sequence. This gives you a symmetric structure. 

*Note:* You can make use of deferred expressions, such as `at='0.25 * lq'`: this evaluates each time the length `lq` gets updated in the environment `env`.

In [ ]:
JC_fodo_arc = env.new_line(name='JC_fodo_arc', length='lcell', components=[
    env.place('mqf', at='0.25 * lq'), # note the deferred expression
    env.place('msf', at= # fill in here , 
    env.place('mb', at='0.25 * lcell'), 
    # fill in here
])

Define the phase advance of the basic cell:

In [ ]:
# Tunes are given in units of [2 pi]

tuneArcCell_X = # fill in here
tuneArcCell_Y = # fill in here

env['tuneArcCell_X'] = tuneArcCell_X
env['tuneArcCell_Y'] = tuneArcCell_Y

Define the reference particle:

In [ ]:
p_ref = xt.Particles('electron', energy0=Ebeam)
env.set_particle_ref(p_ref)

Determine equilibrium optics:

In [ ]:
tw = JC_fodo_arc.twiss(method='4d')

In [ ]:
print (
    f"Qx = {tw.qx:.4f}\n"
    f"Qy = {tw.qy:.4f}"
)

### <span style="color:blue;">Check the tunes `qx` and `qy` in the TWISS summary table. Do they fit your expectation?</span>

### <span style="color:blue;">g) Match the phase advance of your basic arc cell</span>

In [ ]:
opt = JC_fodo_arc.match(
    method='4d', 
    vary=[
        xt.VaryList([
                # fill in here
            ], step=1e-8, tag='quad')
    ], 
    targets=[
        xt.TargetSet(
            qx= # fill in here ,
            qy= # fill in here , 
            tol= # fill in here ,
            tag='tune')
    ], 
    solve=False
)

In [ ]:
opt.solve();

Check the matching result by running the twiss command:

In [ ]:
tw = JC_fodo_arc.twiss(method='4d')

In [ ]:
print (
    f"Qx = {tw.qx:.4f}\n"
    f"Qy = {tw.qy:.4f}"
)

In [ ]:
tw.plot('betx bety');

In [ ]:
tw.plot('dx dy');

#### Calculate minimum and maximum beta function of your cell analytically

### <span style="color:blue;">Compare the maximum and minimum values of beta functions of xsuite to the analytically calculated values. </span>
    
The equations are given on the lecture slides.

In [ ]:
beta_max = # fill in here
beta_min = # fill in here

print(beta_max, beta_min, tw['betx'].max(), tw['betx'].min())

$\implies$ Do these results meet your expectations?

Compare the maximum and minimum values of the dispersion function of xsuite to the analytically calculated values. They are given by: 

$$\hat{D} = \frac{L^2}{\rho}\frac{(1+\frac{1}{2}\sin{(\mu/2)})}{4\sin^2{(\mu/2)}} \text{ and } \check{D} = \frac{L^2}{\rho}\frac{(1-\frac{1}{2}\sin{(\mu/2)})}{4\sin^2{(\mu/2)}}$$

In [ ]:
rho = LB / np.arcsin(THB)

D_max = # fill in here
D_min = # fill in here

print(D_max, D_min, tw['dx'].max(), tw['dx'].min())

$\implies$ Do these results meet your expectations?

### <span style="color:blue;">h) Match the chromaticity of your basic arc cell to zero.</span>

Check the chromaticities `dqx` and `dqy` from the recent Twiss computation. What does this mean?

How can you correct for this, and what value would you set as a target for the chromaticity?

In [ ]:
print (
    f"Q'x = {tw.dqx:.4f}\n"
    f"Q'y = {tw.dqy:.4f}"
)

In [ ]:
opt = JC_fodo_arc.match(
    method='4d', 
     # fill in here ,
    solve=False
)

In [ ]:
opt.solve();

In [ ]:
tw = JC_fodo_arc.twiss(method='4d')

In [ ]:
print (
    f"Q'x = {tw.dqx:.4f}\n"
    f"Q'y = {tw.dqy:.4f}"
)

Have a look at the optics again. 

### <span style="color:blue;">Can you explain why the sextupole strengths for the 'defocusing' sextupole are larger than for the 'focusing' one?</span>

In [ ]:
print (
    f'k2sf = {env.vv['k2sf']:+.3f} 1/m^3\n'
    f'k2sd = {env.vv['k2sd']:+.3f} 1/m^3'
)

In [ ]:
tw.plot('betx bety');

In [ ]:
tw.plot('dx dy');

### <span style="color:blue;">i) Build a full ring with your basic cells.</span>

### <span style="color:blue;">How many cells do you need to close the ring? </span> 

Is the overall length below the required maximum length?

In [ ]:
# Number of cells required
numberOfCells = round(np.pi * 2 / THB / 2) # note the final /2, there are two dipoles per cell
# Circumference of ring
circumference = numberOfCells * Lcell

numberOfCells, circumference

In [ ]:
env['numberofcells'] = # fill in here
env['l_jc_ring'] = 'numberofcells * lcell' # deferred expression

Define a ring consisting of your FODO cells:

*Hint:* You can multiply cells in xsuite line definitions.

[Building lines in the xsuite manual](https://xsuite.readthedocs.io/en/latest/environment.html#lines)

In [ ]:
JC_ring = numberOfCells * JC_fodo_arc

In [ ]:
tw = JC_ring.twiss(method='4d')

In [ ]:
print (
    f"Qx = {tw.qx:.4f}\n"
    f"Qy = {tw.qy:.4f}\n"
    f"Q'x = {tw.dqx:.4f}\n"
    f"Q'y = {tw.dqy:.4f}"
)

### <span style="color:blue;">Do the tunes fit your expectation?</span>

In [ ]:
tw.plot('betx bety')
plt.xlim(0, 500);

### <span style="color:blue;">Check if the ring is closed. </span>

If necessary adjust the bending angle accordingly.

In [ ]:
survey = JC_ring.survey()

In [ ]:
survey.plot();

In [ ]:
print(survey['theta'][-1], 2 * np.pi)

### <span style="color:blue;">j) Calculate the synchrotron radiation integrals using the `radiation_integrals` option of the Twiss command.</span>

In [ ]:
tw = JC_ring.twiss(method='4d', radiation_integrals=True)

You can access the results of the twiss table like this:

In [ ]:
print (
    f"I_2  = {tw.rad_int_i2:.4e}\n"
    f"I_5x = {tw.rad_int_i5x:.4e}"
)

### <span style="color:blue;">Calculate equilibrium emittance and energy loss per turn using the synchrotron radiation integrals. </span>

Do you obtain the emittance you were designing for?

In [ ]:
epsilon_x_SRI = # fill in here
U0_SRI = # fill in here

print (
    f"epsn_x,SRI = {epsilon_x_SRI:.3e} m.rad\n"
    f"U_0,SRI    = {U0_SRI:.3f} GeV"
)

$\,$

$\,$

## 2. Dispersion suppressors and straight sections

### <span style="color:blue;">a) Design a dispersion suppressor section (DS) for both **start (<span style="color:red;">left, "DSL"</span>)** and **end (<span style="color:red;">right, "DSR"</span>)** of the arc sections of your storage ring and implement it in xsuite.  </span>
    
What scheme do you use and why? Why is it not possible to use the identical section on both sides of an arc?

In [ ]:
# Define quadrupole strengths of the dispersion suppressor
env['k1qfds1'] = env['k1qf']
env['k1qdds1'] = env['k1qd']
env['k1qfds2'] = env['k1qf']
env['k1qdds2'] = env['k1qd']

In [ ]:
env.new('mbds', # fill in here )
env.new('mqfds1', # fill in here )
env.new('mqdds1', # fill in here )
env.new('mqfds2', # fill in here )
env.new('mqdds2', # fill in here );

Define individual sequences for your DSL and DSR.

In [ ]:
JC_dsl = env.new_line(name='JC_dsl', length='lds', components=[
    # fill in here
])

In [ ]:
JC_dsr = env.new_line(name='JC_dsr', length='lds', components=[
    # fill in here
])

In [ ]:
env['lds'] = '2 * lcell' # deferred expression

Define a small sequence to illustrate the features of the dispersion suppressor (DS). Use the following layout, arriving from an **arc's right end** (DSR) into the beginning of a straight section before the **left side of the next arc** (DSL): 

`arc cell` + ... + `arc cell` + **`DSR`** + `marker` + **`DSL`** + `arc cell` + ... + `arc cell`

In [ ]:
env.new('mark_endds', xt.Marker);

In [ ]:
illustrate_DS = env.new_line(name='illustrate_DS', length='8 * lcell', components=[
    env.place(JC_fodo_arc, at='0.5 * lcell'), 
    env.place(JC_fodo_arc, at='1.5 * lcell'), 
    env.place(JC_dsr, at= # fill in here ), 
    env.place('mark_endds', at= # fill in here ), 
    env.place(JC_dsl, at= # fill in here ), 
    env.place(JC_fodo_arc, at= # fill in here ), 
    env.place(JC_fodo_arc, at= # fill in here )
])

Save start values of periodic solutions as start values for non-periodic matching procedures:

We need to save the optics functions `betx`, `bety`, `alfx`, `alfy`, `dx`, and `dpx` as references for **non-periodic** `twiss` calculations like, for example, to illustrate how the dispersion suppressor works.

In [ ]:
twiss_fodo_arc = JC_fodo_arc.twiss(method='4d')

You can get the values from the python object `twiss_fodo_arc`, which contains the results of the `twiss` calculation above.

In [ ]:
print(
    twiss_fodo_arc['betx'][0],
    twiss_fodo_arc['bety'][0],
    twiss_fodo_arc['alfx'][0],
    twiss_fodo_arc['alfy'][0],
    twiss_fodo_arc['dx'][0],
    twiss_fodo_arc['dpx'][0])

Perform a non-periodic `twiss` calculation with these starting values and have a look at the optics:

In [ ]:
tw = illustrate_DS.twiss(
    method='4d', 
    # note the following explicit initial conditions:
    betx=twiss_fodo_arc['betx'][0], 
    bety=twiss_fodo_arc['bety'][0], 
    alfx=twiss_fodo_arc['alfx'][0], 
    alfy=twiss_fodo_arc['alfy'][0], 
    dx=twiss_fodo_arc['dx'][0], 
    dpx=twiss_fodo_arc['dpx'][0]
)

In [ ]:
tw.plot('betx bety');

In [ ]:
tw.plot('dx dy');

Value of the dispersion function at the end of the dispersion suppressor (center of this sequence):

In [ ]:
print (tw['dx'][tw['name'] == 'mark_endds']) # in [m]

### <span style="color:blue;">b) Define a straight cell for the straight sections </span>

In [ ]:
env['k1qfs'] = # fill in here
env['k1qds'] = # fill in here

In [ ]:
env.new('mqfs',  # fill in here )
env.new('mqds',  # fill in here );

In [ ]:
JC_fodo_ss = env.new_line(name='JC_fodo_ss', length='lcell', components=[
    # fill in here
])

Calculate the optics of the straight cell. If necessary, re-match the optics.

In [ ]:
twiss_fodo_ss = JC_fodo_ss.twiss(method='4d')

In [ ]:
twiss_fodo_ss.plot('betx bety');

In [ ]:
twiss_fodo_ss.plot('dx dy');

### <span style="color:blue;">c) Define matching sections (MS) for the beginning (<span style="color:red;">left, "MSL"</span>) and the end (<span style="color:red;">right, "MSR"</span>) of your straight sections. </span>
    
Why do you need matching sections? How many parameters do you need to match? How many degrees of freedoms (= quadrupoles) do you need? 

In [ ]:
env['k1qfms1'] = # fill in here
env['k1qdms1'] = # fill in here
# fill in here

In [ ]:
env.new('mqfms1', # fill in here )
env.new('mqdms1', # fill in here )
# fill in here

*Comment:* If you use quadrupoles with half the length be careful about the type of quadrupole at the beginning and at the end of the matching section. They might belong to a regular cell or the dispersion suppressor. Choose the according correct quadrupole element.

In [ ]:
env['lms'] = # fill in here # length of the matching section in [m]

In [ ]:
JC_msl = env.new_line(name='JC_msl', length='lms', components=[
    # fill in here
])

In [ ]:
JC_msr = env.new_line(name='JC_msr', length='lms', components=[
    # fill in here
])

Define a small symmetric sequence to **match** the optics at the transition from the periodic solution in the arc to the one in the straight section. (Why do we need to match?)

Use the following structure: 

`arc cell` + **`DSR` + `MSL`** + `straight cell` + ... + `straight cell` + **`MSR` + `DSL`** + `arc cell`

In [ ]:
env.new('mark_fodo_ss', xt.Marker);
env.new('mark_fodo_arc', xt.Marker);

In [ ]:
match_MS = env.new_line(name='match_MS', length='lcell * # fill in here', components=[
    # fill in here
    env.place(JC_dsr, at= # fill in here ),
    env.place(JC_msl, at= # fill in here ),
    env.place('mark_fodo_ss', at= # fill in here ), # marker at the end of the matching section before the straight section cells
    # fill in here
    env.place('mark_fodo_arc', at= # fill in here ), # marker at the end of the dispersion suppressor before the arc cells
    # fill in here
])

Start the optics calculation with values at the begining of the arc fodo cell

In [ ]:
print(
    twiss_fodo_arc['betx'][0], 
    twiss_fodo_arc['bety'][0], 
    twiss_fodo_arc['alfx'][0], 
    twiss_fodo_arc['alfy'][0],
    twiss_fodo_arc['dx'][0], 
    twiss_fodo_arc['dpx'][0]
)

In [ ]:
tw = match_MS.twiss(
    method='4d', 
    # note the following explicit initial conditions:
    betx=twiss_fodo_arc['betx'][0], 
    bety=twiss_fodo_arc['bety'][0], 
    alfx=twiss_fodo_arc['alfx'][0], 
    alfy=twiss_fodo_arc['alfy'][0],
    dx=twiss_fodo_arc['dx'][0], 
    dpx=twiss_fodo_arc['dpx'][0]
)

In [ ]:
tw.plot('betx bety');

In [ ]:
tw.plot('dx dy');

Match the optics. Where do you have to fulfill which constraints? 

Confirm that the markers `mark_fodo_arc` and `mark_fodo_ss` are installed in your sequence to retrieve that information!

In [ ]:
opt = match_MS.match(
    method='4d', 
    betx=twiss_fodo_arc['betx'][0], 
    bety=twiss_fodo_arc['bety'][0], 
    alfx=twiss_fodo_arc['alfx'][0], 
    alfy=twiss_fodo_arc['alfy'][0],
    dx=twiss_fodo_arc['dx'][0], 
    dpx=twiss_fodo_arc['dpx'][0], 
    vary=[
        xt.VaryList([
            # fill in here
        ], step=1e-3, tag='ds_ms')
    ], 
    targets=[
        xt.TargetSet(
            # fill in here ,
            at='mark_fodo_ss', 
            tol=1e-6
        ), 
        xt.TargetSet(
            # fill in here ,
            at='mark_fodo_arc', 
            tol=1e-6
        ), 
    ], 
    solve=False, 
    assert_within_tol=False, 
    restore_if_fail=False
)

In [ ]:
opt.solve();

In [ ]:
tw = match_MS.twiss(
    method='4d', 
    # note the following explicit initial conditions:
    betx=twiss_fodo_arc['betx'][0], 
    bety=twiss_fodo_arc['bety'][0], 
    alfx=twiss_fodo_arc['alfx'][0], 
    alfy=twiss_fodo_arc['alfy'][0],
    dx=twiss_fodo_arc['dx'][0], 
    dpx=twiss_fodo_arc['dpx'][0]
)

In [ ]:
tw.plot('betx bety');

In [ ]:
tw.plot('dx dy');

Confirm the matching result with a periodic `twiss` calculation.

In [ ]:
tw = match_MS.twiss(method='4d')

In [ ]:
tw.plot('betx bety');

In [ ]:
tw.plot('dx dy');

### <span style="color:blue;">d) Include straight sections at four places in your ring</span>

How do you distribute the straight sections?

In [ ]:
print (numberOfCells / 4)

In [ ]:
env['numberofarccells'] = '0.25 * numberofcells'
env['numberofstraightcells'] = # fill in here

Define sequences for your arcs, straight sections and the full ring. As this ring becomes very long, also define a sequence for one quarter of the ring to speed up matching routines (one straight section and one arc).

We use the following structure to define the arcs: <br />`DSL` + `arc cell` + ... + `arc cell` + `DSR`

We use the following structure to define the straight sections: <br />`MSL` + `straight cell` + ... + `straight cell` + `MSR`

In [ ]:
JC_arc = # fill in here

In [ ]:
JC_ss = # fill in here

In [ ]:
JC_quarter = JC_ss + JC_arc

In [ ]:
JC = 4 * JC_quarter

In [ ]:
tw = JC.twiss(method='4d')

In [ ]:
tw.plot('betx bety');
# plt.xlim(17000, 25000);

In [ ]:
tw.plot('dx dy');
# plt.xlim(17000, 25000);

In [ ]:
survey = JC.survey()

In [ ]:
survey.plot();

### <span style="color:blue;">e) Observe tunes and chromaticities. Do they match your expectation? Re-match tunes and chromaticities.</span>

In [ ]:
print (
    f"Qx = {tw.qx:.4f}\n"
    f"Qy = {tw.qy:.4f}\n"
    f"Q'x = {tw.dqx:.4f}\n"
    f"Q'y = {tw.dqy:.4f}"
)

First, the tunes need to be matched to avoid the proximity of any major resonances. As this storage ring is very long, the quadrupole strengths of the arc cells could be used to do the adjustment. If the phase advance of those cells is critical and should not be touched the quadrupoles of the straight cells can be used. 

We want to match to the following tunes:

In [ ]:
tuneToBeX = # fill in here
tuneToBeY = # fill in here

env['tunetobex'] = tuneToBeX
env['tunetobey'] = tuneToBeY

And we also want to match the chromaticity once more.

First, switch off the sextupoles and calculate the natural (absolute) chromaticity of your storage ring. Can you confirm the relation $dQ/d\delta\approx - Q$?

In [ ]:
env.vv['k2sf'] = 0
env.vv['k2sd'] = 0

In [ ]:
tw = JC.twiss(method='4d')

In [ ]:
print (
    f"Qx = {tw.qx:.4f}\n"
    f"Qy = {tw.qy:.4f}\n"
    f"Q'x = {tw.dqx:.4f}\n"
    f"Q'y = {tw.dqy:.4f}"
)

Let us rematch the tunes and chromaticities. What value do you choose and why? (Think about instabilities and collective effects.)

In [ ]:
opt = JC.match(
    method='4d', 
    vary=[
        xt.VaryList(['k1qf', 'k1qd'], step=1e-8, tag='quad'), 
        xt.VaryList(['k2sf', 'k2sd'], step=1e-3, tag='sext')
    ], 
    targets=[
        xt.TargetSet(
            qx= # fill in here ,
            qy= # fill in here ,
            tol=1e-10, tag='tune'), 
        xt.TargetSet(
            dqx= # fill in here ,
            dqy= # fill in here ,
            tol=1e-6, tag='chroma')
    ], 
    solve=False
)

In [ ]:
opt.solve();

In [ ]:
tw = JC.twiss(method='4d')

In [ ]:
print (
    f"Qx = {tw.qx:.4f}\n"
    f"Qy = {tw.qy:.4f}\n"
    f"Q'x = {tw.dqx:.4f}\n"
    f"Q'y = {tw.dqy:.4f}"
)

### <span style="color:blue;">f) Calculate and save the synchrotron radiation integrals of your storage ring for analytical calculations later</span>

In [ ]:
tw = JC.twiss(method='4d', radiation_integrals=True)

In [ ]:
SRI1 = tw.rad_int_i1x
SRI2 = tw.rad_int_i2
SRI3 = tw.rad_int_i3
SRI4 = tw.rad_int_i4
SRI5 = tw.rad_int_i5x

In [ ]:
print(SRI1, SRI2, SRI3, SRI4, SRI5)

$\,$

$\,$

## If time permits: 3. RF sections

## <span style="color:#af0000;">Let it shine! ;-)</span>

### <span style="color:blue;">a) Switch on radiation effects.</span>

Let us now switch on synchrotron radiation effects:

In [ ]:
JC.configure_radiation(model='mean')

Observe how this impacts the orbit and optics for a non-periodic TWISS solution, using the non-radiating solution as initial condition. 

Can you explain what happens to the horizontal orbit?

In [ ]:
tw = JC.twiss(
    method='6d', 
    # note the following explicit initial conditions:
    betx=tw['betx'][0], 
    bety=tw['bety'][0], 
    alfx=tw['alfx'][0], 
    alfy=tw['alfy'][0],
    dx=tw['dx'][0], 
    dpx=tw['dpx'][0],
)

In [ ]:
tw.plot('x', lattice=False);

And what about tunes and optics?

In [ ]:
print (
    f"Qx = {tw.mux[-1]:.4f}\n"
    f"Qy = {tw.muy[-1]:.4f}"
)

In [ ]:
tw.plot('betx bety', lattice=False);

In [ ]:
tw.plot('dx', lattice=False);

An RF system will be needed to compensate for these radiation effects. This means, RF cavities need to be defined and installed in the lattice.
In this exercise we use parameters from the RF system designed for FCC-ee. It uses 400 MHz cavities that consist of four cells with a length of 0.375 m each.

Compute the overall voltage to guarantee stable operation.

The energy gain in the cavity is given by 

$$U = eV_{RF} \sin{(2\pi (\phi-hf_0t))},$$

where $\phi$ is the phase lag ("synchronous phase"), $h$ the harmonic number of the ring, and  $f_0$ the cavity frequency.

### <span style="color:blue;">b) Estimate the total RF voltage which is needed to compensate the synchrotron radiation energy loss.</span>

In [ ]:
VRF = # fill in here # total rf voltage in [V]
LGRF = # fill in here # synchrotron phase in [degree]

### <span style="color:blue;">c) Define a straight cell and straight sections that contain RF cavities (please check if the definition is consistent to yours):</span>

In [ ]:
env['switch_rf_on'] = 0
env['vrf'] = VRF
env['lgrf'] = LGRF
env['numberofcav400'] = int(env['numberofstraightcells']) * 4 * 2 # 4 straight sections x 4 cavities / module x 2 modules / FODO cell
env['lcavcell400'] = 0.375
env['harmonic400'] = 114746
env['numberofcellscav400'] = 4
env['vcell400'] = 'vrf / numberofcav400 / numberofcellscav400 * switch_rf_on' # deferred expression
env['lgrf400'] = '(switch_rf_on * lgrf + (1 - switch_rf_on) * 0.5 * 360)' # deferred expression

In [ ]:
env.new('cavcell400', xt.Cavity, length='lcavcell400', voltage='vcell400', lag='lgrf400', harmonic='harmonic400');

In [ ]:
CavModule = env.new_line(
    name='cavmodule', 
    length='numberofcellscav400 + 0.5 * (numberofcellscav400 - 1) * lcavcell400', 
    components=[
        env.place('cavcell400', at='0.5 * lcavcell400'), 
        env.place('cavcell400', at='2.0 * lcavcell400'), 
        env.place('cavcell400', at='3.5 * lcavcell400'), 
        env.place('cavcell400', at='5.0 * lcavcell400')
    ])

In [ ]:
JC_fodo_ss_RF = env.new_line(name='JC_fodo_ss_RF', length='lcell', components=[
    env.place('mqfs', at='0.25 * lq'), 
    env.place(CavModule, at='0.25 * lcell'), 
    env.place('mqds', at='0.5 * lcell - 0.25 * lq'), 
    env.place('mqds', at='0.5 * lcell + 0.25 * lq'), 
    env.place(CavModule, at='0.75 * lcell'), 
    env.place('mqfs', at='lcell - 0.25 * lq')
])

Define a straight section and the full collider lattice as above but including RF cavities:

In [ ]:
JC_ss_RF = # fill in here

In [ ]:
JC_quarter_RF = JC_ss_RF + JC_arc

In [ ]:
JC_RF = 4 * JC_quarter_RF

### <span style="color:blue;">d) Switch on radiation and observe tunes and chromaticities</span>

In [ ]:
JC_RF.configure_radiation(model='mean')

Switch on the RF system.

In [ ]:
env['switch_rf_on'] = 1

First we have to match the energy gain in our cavities to make sure we receive as much energy as we lose due to synchrotron radiation:

(Start with with a phase lag of 180 degrees, otherwise calculate the required value you need from the equation for the energy gain.)

In [ ]:
env['lgrf'] = # fill in here

In [ ]:
# printing existing cavities in the rf-enabled JC:

JC_RF.get_table(attr=True).rows.match(element_type='Cavity').cols['voltage frequency lag harmonic']

Try a first `twiss` calculation including radiation effects passing `eneloss_and_damping=True` to the twiss command:

In [ ]:
tw = JC_RF.twiss(method='6d', eneloss_and_damping=True)

In [ ]:
print (
    f"Qx = {tw.qx:.4f}\n"
    f"Qy = {tw.qy:.4f}\n"
    f"Q'x = {tw.dqx:.4f}\n"
    f"Q'y = {tw.dqy:.4f}"
)

What happened now to the tunes, chromaticities and the optics? Can you explain?

In [ ]:
tw.plot('betx bety', lattice=False);
# plt.xlim(17000, 25000);

In [ ]:
tw.plot('dx dy', lattice=False);
# plt.xlim(17000, 25000);

Let us rematch tunes and chromaticities.

In [ ]:
opt = JC_RF.match(
    method='6d', 
    vary=[
        xt.VaryList(['k1qf', 'k1qd'], step=1e-8, tag='quad'), 
        xt.VaryList(['k2sf', 'k2sd'], step=1e-3, tag='sext')
    ], 
    targets=[
        xt.TargetSet(qx=env['tunetobex'], qy=env['tunetobey'], tol=1e-8, tag='tune'), 
        xt.TargetSet(dqx=2.0, dqy=2.0, tol=1e-3, tag='chroma')
    ], 
    solve=False
)

In [ ]:
opt.solve();

In [ ]:
tw = JC_RF.twiss(method='6d', eneloss_and_damping=True)

In [ ]:
print (
    f"Qx = {tw.qx:.4f}\n"
    f"Qy = {tw.qy:.4f}\n"
    f"Q'x = {tw.dqx:.4f}\n"
    f"Q'y = {tw.dqy:.4f}"
)

In [ ]:
tw.plot('betx bety', lattice=False);
# plt.xlim(17000, 25000);

In [ ]:
tw.plot('dx dy', lattice=False);
# plt.xlim(17000, 25000);

### <span style="color:blue;">e) Observe the transverse orbit. Can you explain the pattern you see?</span>

In [ ]:
tw.plot('x', lattice=False);
# plt.xlim(17000, 25000);

This effect has already been observed in LEP measurements at the time:

![sawtooth orbit](sawtooth.png)

(Image obtained from arXiv (https://arxiv.org/abs/hep-ex/0410026), it has been published as Fig. 8 in: Eur. Phys. J. C 39, 253–292 (2005). https://doi.org/10.1140/epjc/s2004-02108-8 )

### <span style="color:blue;">f) Energy tapering

To compensate for this sawtooth orbit and related radiation effects on the optics, the local reference momentum can be adjusted to compensate for the synchrotron radiation loss.</span>

In [ ]:
# xsuite's straight forward energy loss compensation will only work 
# if there are no multiple instances of the same element:
JC_RF.replace_all_repeated_elements()
JC_RF.build_tracker()

JC_RF.compensate_radiation_energy_loss()

In [ ]:
tw = JC_RF.twiss(method='6d', eneloss_and_damping=True)

In [ ]:
tw.plot('x', lattice=False)
plt.ylim(-1e-3, 1e-3);
# plt.xlim(17000, 25000);

In [ ]:
tw.plot('betx bety', lattice=False);
# plt.xlim(17000, 25000);

In [ ]:
tw.plot('dx dy', lattice=False);
# plt.xlim(17000, 25000);

In [ ]:
print (
    f"Qx = {tw.qx:.4f}\n"
    f"Qy = {tw.qy:.4f}\n"
    f"Q'x = {tw.dqx:.4f}\n"
    f"Q'y = {tw.dqy:.4f}"
)

### <span style="color:blue;">g) Compute equilibrium beam parameters (using the xsuite `twiss` command with `eneloss_and_damping=True`)</span>

In [ ]:
print (tw.eq_gemitt_x, tw.eq_gemitt_y)

Compare the results to the analytical values from the calculation using the synchrotron radiation integrals above:

In [ ]:
epsilon_x_SRI = C_q * gamma_L**2 / J_x * SRI5 / SRI2
U0_SRI = 8.846e-5 / 2 / np.pi * 175**4 * SRI2 # in GeV
alpha_c_SRI = SRI1 / tw.circumference
dp_p_SRI = np.sqrt(C_q * gamma_L**2 * SRI3 / (2 * SRI2 + SRI4))
J_x_SRI = 1 - SRI4 / SRI2
J_s_SRI = 2 + SRI4 / SRI2

print(epsilon_x_SRI, U0_SRI, alpha_c_SRI, dp_p_SRI, J_x_SRI, J_s_SRI)

### <span style="color:blue;">h) How many particles and bunches can you fill into the ring before you reach the limit of synchrotron radiation power?</span>

Number of total particles:

In [ ]:
"%e" % ( # fill in here )